<a href="https://colab.research.google.com/github/Minh-A/ARIMA_Dacon/blob/main/ARIMA_Dacon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [ ]:
# 구글 코랩 연결
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib as mpl
import matplotlib.pyplot as plt 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# 날짜 처리 ARIMQ용
data1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/여름방학 세미나/input/201901-202003.csv')
data1 = data1.fillna('')

In [ ]:
data1

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3
...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4
24697788,202003,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3
24697789,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6
24697790,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6


In [ ]:
# 2020 4월 데이터
data11 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/여름방학 세미나/input/202004.csv')
data11 = data11.fillna('')

In [ ]:
data11

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,3,779600,4
1,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,2,3,14,2427700,15
2,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,50s,1,4,18,2184600,18
3,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,50s,2,4,21,3653100,23
4,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,60s,1,5,15,2765500,18
...,...,...,...,...,...,...,...,...,...,...,...,...
1350317,202004,충북,충주시,휴양콘도 운영업,충북,충주시,20s,1,1,5,77000,5
1350318,202004,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,6,92000,6
1350319,202004,충북,충주시,휴양콘도 운영업,충북,충주시,40s,2,3,5,193000,5
1350320,202004,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,5,86000,7


# 데이터 셋 생성

In [ ]:
# 데이터1 정제
data1_1 = data1.drop(['CARD_CCG_NM', 'HOM_CCG_NM','FLC','HOM_SIDO_NM','AGE','SEX_CTGO_CD','CSTMR_CNT','CNT'], axis=1)

columns = ['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM']
data1_1 = data1_1.groupby(columns).sum().reset_index(drop=False)
data1_1

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,148354746
1,201901,강원,골프장 운영업,1229200312
2,201901,강원,과실 및 채소 소매업,1286979106
3,201901,강원,관광 민예품 및 선물용품 소매업,30656770
4,201901,강원,그외 기타 스포츠시설 운영업,13101500
...,...,...,...,...
9433,202003,충북,피자 햄버거 샌드위치 및 유사 음식점업,1315245299
9434,202003,충북,한식 음식점업,16152482704
9435,202003,충북,호텔업,15248550
9436,202003,충북,화장품 및 방향제 소매업,428881434


In [ ]:
# 데이터2 정제
data11_1 = data11.drop(['CARD_CCG_NM', 'HOM_CCG_NM','FLC','HOM_SIDO_NM','AGE','SEX_CTGO_CD','CSTMR_CNT','CNT'], axis=1)

columns = ['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM']
data11_1 = data11_1.groupby(columns).sum().reset_index(drop=False)
data11_1

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,88823988
1,202004,강원,골프장 운영업,4708346820
2,202004,강원,과실 및 채소 소매업,1121028924
3,202004,강원,관광 민예품 및 선물용품 소매업,14360780
4,202004,강원,그외 기타 스포츠시설 운영업,227200
...,...,...,...,...
605,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1373635928
606,202004,충북,한식 음식점업,18911036160
607,202004,충북,호텔업,14121500
608,202004,충북,화장품 및 방향제 소매업,450507431


In [ ]:
# 데이터1과 데이터2를 합치기
data2 = pd.concat([data1_1, data11_1], axis=0)
data2 = data2.reset_index(drop=True)
data2

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,148354746
1,201901,강원,골프장 운영업,1229200312
2,201901,강원,과실 및 채소 소매업,1286979106
3,201901,강원,관광 민예품 및 선물용품 소매업,30656770
4,201901,강원,그외 기타 스포츠시설 운영업,13101500
...,...,...,...,...
10043,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1373635928
10044,202004,충북,한식 음식점업,18911036160
10045,202004,충북,호텔업,14121500
10046,202004,충북,화장품 및 방향제 소매업,450507431


In [ ]:
# 제공 데이터 인코딩
dtypes = data2.dtypes
encoders = {}
for column in data2.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(data2[column])
        encoders[column] = encoder
        
for column in encoders.keys():
    encoder = encoders[column]
    data2[column] = encoder.transform(data2[column])

In [ ]:
data2

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,0,0,148354746
1,201901,0,1,1229200312
2,201901,0,2,1286979106
3,201901,0,3,30656770
4,201901,0,5,13101500
...,...,...,...,...
10043,202004,16,36,1373635928
10044,202004,16,37,18911036160
10045,202004,16,38,14121500
10046,202004,16,39,450507431


In [ ]:
# 템플릿 생성
REG_YYMMs     = data2['REG_YYMM'].unique()
CARD_SIDO_NMs = data2['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = data2['STD_CLSS_NM'].unique()
STD_CLSS_NMs.sort()                                    # 매장 순서정렬을 위한 코드

temp = []
for REG_YYMM in REG_YYMMs:
    for CARD_SIDO_NM in CARD_SIDO_NMs:
        for STD_CLSS_NM in STD_CLSS_NMs:
            temp.append([REG_YYMM, CARD_SIDO_NM, STD_CLSS_NM])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns= ['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])

In [ ]:
temp

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM
0,201901,0,0
1,201901,0,1
2,201901,0,2
3,201901,0,3
4,201901,0,4
...,...,...,...
11147,202004,16,36
11148,202004,16,37
11149,202004,16,38
11150,202004,16,39


In [ ]:
# 기초 데이터셋 생성
daset = pd.merge(temp, data2, how= 'left', left_on = ('REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'), right_on = ('REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'))

In [ ]:
daset

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,0,0,1.483547e+08
1,201901,0,1,1.229200e+09
2,201901,0,2,1.286979e+09
3,201901,0,3,3.065677e+07
4,201901,0,4,NaN
...,...,...,...,...
11147,202004,16,36,1.373636e+09
11148,202004,16,37,1.891104e+10
11149,202004,16,38,1.412150e+07
11150,202004,16,39,4.505074e+08


In [ ]:
# 결측치에 0을 넣기
daset=daset.fillna(0)
daset

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,0,0,1.483547e+08
1,201901,0,1,1.229200e+09
2,201901,0,2,1.286979e+09
3,201901,0,3,3.065677e+07
4,201901,0,4,0.000000e+00
...,...,...,...,...
11147,202004,16,36,1.373636e+09
11148,202004,16,37,1.891104e+10
11149,202004,16,38,1.412150e+07
11150,202004,16,39,4.505074e+08


In [ ]:
# 메인 키값 설정
daset['key'] = daset['CARD_SIDO_NM']*100 + daset['STD_CLSS_NM']
daset["AMT"] = np.log1p(daset['AMT'])
daset = daset[["key","AMT"]]
daset

,key,AMT
0,0,18.815117
1,1,20.929630
2,2,20.975564
3,3,17.238364
4,4,0.000000
...,...,...
11147,1636,21.040727
11148,1637,23.663012
11149,1638,16.463209
11150,1639,19.925885


# 예측 코드

## 5월 예측 템플릿 만들기

In [ ]:
# 템플릿 만들기
newdata5 = daset.loc[daset['key']== 4]
newdata5 = newdata5.reset_index(drop= True)

In [ ]:
newdata5

,key,AMT
0,4,0.000000
1,4,0.000000
2,4,0.000000
3,4,0.000000
4,4,12.083911
5,4,0.000000
6,4,0.000000
7,4,0.000000
8,4,0.000000
9,4,0.000000


In [ ]:
# 템플릿 분할
newdata6 = newdata5.copy()
newdata7 = newdata5.copy()

In [ ]:
# 5월 예측 템플릿
for aa in range(17):
  for i in range(41):
    num = num = aa*100 + i
    print(num)
    ii = daset.loc[daset['key']==num]
    ii = ii.reset_index(drop= True)
    if ii["AMT"][0] == 0:
      print("결측치 있어")
      ii.loc[10000] = [num, 0]
      newdata5 = pd.concat([newdata5, ii], axis=0)
    else:  
      arma = ARIMA(ii["AMT"], order = (0,0,0))
      arima = arma.fit(trend = 'c', full_output = True, disp=1)
      bb = arima.forecast(steps=1)
      print(bb[0][0])
      ii.loc[10000] = [num, bb[0][0]]
      newdata5 = pd.concat([newdata5, ii], axis=0)

0
18.558410553484975
1
21.79319827351228
2
20.814152763143834
3
17.11671080310362
4
결측치 있어
5
15.400165000196209
6
19.952180552286187
7
22.7488559642487
8
14.830350722465166
9
20.796819475358262
10
19.5493899074469
11
22.743151655309656
12
14.023645320195708
13
17.310032316555162
14
4.058395091791018
15
11.999771318497514
16
20.89353176962436
17
20.86981528777813
18
21.50282185591142
19
20.258212788764222
20
23.045994754954993
21
21.749241438204887
22
20.29435884398926
23
18.46007930945157
24
19.129954221444663
25
21.301788951305394
26
20.963144178325685
27
21.3249521390982
28
15.005472423415519
29
19.88913958580718
30
결측치 있어
31
21.064845705942325
32
20.13071180734853
33
23.547327536722012
34
22.974322571004443
35
10.402349084263852
36
21.134585716739746
37
23.907418162476755
38
20.426659786205217
39
20.19790100894553
40
21.347232860090994
100
21.053941328638643
101
23.031131343759
102
22.678177666075044
103
18.655676765776775
104
결측치 있어
105
17.340063824760712
106
22.9272738601401
107
2

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


22.24392584821616
202
21.752825650588843
203
17.69872608968206
204
13.098488188108767
205
15.949782106125458
206
20.775594289846556
207
23.81904598381382
208
14.703299201290427
209
21.3426353058073
210
20.303475245058785
211
23.125261854813772
212
17.050347038852514
213
17.702673265205632
214
결측치 있어
215
15.940661522062634
216
21.823786281528506
217
21.913639878107364
218
22.237168963367157
219
20.603063434754628
220
23.909346366421808
221
22.24600264030242


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


222
20.67481296257525
223
13.387978596090695
224
20.803054818099497
225
22.266551744859623
226
22.262138850652654
227
21.597166469973068
228
14.205446506736772
229
18.277315545688765
230
결측치 있어
231
21.668917786949663
232
20.923091117389077
233
24.50166105081674
234
23.706646593975663
235
16.7536524611533
236
22.151424718552
237
24.768848695487765
238
17.94230053813304
239
21.259537091673618
240
19.71311554435218
300
19.198336808792018
301
22.219084812748555
302
21.37565171572243
303
17.74913158766768
304
3.7177237833614987
305
15.356547058555066
306
20.157832235124236
307
23.36469011803328
308
13.894344778494547
309
20.927058785321076
310
19.696251221503744
311
22.83696343277159
312
12.318367222839566
313
16.632673114733283
314
결측치 있어
315
결측치 있어
316
21.39735557767414
317
21.481466053204738
318
21.749803512102172
319
20.913980619013763
320
23.85950791208823
321
22.249745261751027
322
19.968764348536496
323
9.896924699319086
324
20.058929603591853
325
22.313227127507705
326
21.0136302482

...

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


20.669382609736722
1420
23.032889914051008
1421
21.500610753071065
1422
20.137200858354696
1423
17.025293947532788
1424
19.107724431060998
1425
20.74589633118849
1426
21.42156309239372
1427
20.96775401457709
1428
19.92153912728306
1429
19.543137798820975
1430
21.49331708206679
1431
20.58824318174819
1432
19.969393363326912
1433
22.89207786096524
1434
22.61882759016852
1435
결측치 있어
1436
20.75978314191651
1437
23.722764428989805
1438
20.723046384570974
1439
20.1449436743653
1440
19.664177232172545
1500
20.746408568565247
1501
20.8231333114742
1502
21.093754657190544
1503
17.170679249640727
1504
13.512769075825766
1505
15.629688049067354
1506
21.36579946720502
1507
23.14724394245246
1508
15.176132246498861
1509
21.130473450109484
1510
18.957815441742977
1511
22.65936480864774
1512
12.07778746724925
1513
17.43661486086929
1514
결측치 있어
1515
결측치 있어
1516
21.150805591112263
1517
21.422248013739047
1518


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


21.62841819163624
1519
20.972279247792347
1520
23.359415307383113
1521
21.885496268308742
1522
20.08624695817706
1523
16.08537259999717
1524
19.63764717277806
1525
21.72123482835251
1526
20.709569415419594
1527
21.101986789949162
1528
11.3439529746776
1529
17.586056493466057
1530
11.455555974535866
1531
21.500495238974143
1532
20.578830918972592
1533
24.195003163048973
1534
23.320132884506673
1535
6.529705133883361
1536
21.504061946940322
1537
24.12150881755774
1538
18.277867909596388
1539
20.99026401383899
1540
18.998174952883595
1600
18.929168930169904
1601
21.306102248985017
1602
20.40802339884972
1603
16.850073892885906
1604
결측치 있어
1605
15.291961729778127
1606
21.330248525134564
1607
22.820269251919648
1608
14.534571431436843
1609
20.877259212860125
1610
19.074040879770493
1611
21.88235017810175
1612
결측치 있어
1613
16.781326968724308
1614
7.520574905947882
1615
4.7203281263491235
1616
20.65570570115796
1617
20.968923750832335
1618
21.13848532092836
1619
19.277869945555782
1620
23.1880

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [ ]:
# 2020년 5월 예측 템플릿
newdata5 = newdata5.iloc[16:,:].reset_index(drop= True)
newdata5

,key,AMT
0,0.0,18.815117
1,0.0,18.804008
2,0.0,18.510878
3,0.0,18.484111
4,0.0,18.813505
...,...,...
11844,1640.0,19.218517
11845,1640.0,17.667483
11846,1640.0,16.359746
11847,1640.0,16.048576


## 6월 예측 템플릿 만들기

In [ ]:
# 6월 예측
for aa in range(17):
  for i in range(41):
    num = num = aa*100 + i
    ii = newdata5.loc[newdata5['key']==num]
    ii = ii.reset_index(drop= True)
    if ii["AMT"][0] == 0:
      ii.loc[10000] = [num, 0]
      newdata6 = pd.concat([newdata6, ii], axis=0)
    else:  
      arma = ARIMA(ii["AMT"], order = (0,0,0))
      arima = arma.fit(trend = 'c', full_output = True, disp=1)
      bb = arima.forecast(steps=1)
      ii.loc[10000] = [num, bb[0][0]]
      newdata6 = pd.concat([newdata6, ii], axis=0)

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", Conve

In [ ]:
# 2020년 6월 예측 템플릿
newdata6 = newdata6.iloc[16:,:].reset_index(drop= True)
newdata6

,key,AMT
0,0.0,18.815117
1,0.0,18.804008
2,0.0,18.510878
3,0.0,18.484111
4,0.0,18.813505
...,...,...
12541,1640.0,17.667483
12542,1640.0,16.359746
12543,1640.0,16.048576
12544,1640.0,18.504392


## 7월 예측 템플릿 만들기

In [ ]:
# 7월 예측
for aa in range(17):
  for i in range(41):
    num = num = aa*100 + i
    ii = newdata6.loc[newdata6['key']==num]
    ii = ii.reset_index(drop= True)
    if ii["AMT"][0] == 0:
      ii.loc[10000] = [num, 0]
      newdata7 = pd.concat([newdata7, ii], axis=0)
    else:  
      arma = ARIMA(ii["AMT"], order = (0,0,0))
      arima = arma.fit(trend = 'c', full_output = True, disp=1)
      bb = arima.forecast(steps=1)
      ii.loc[10000] = [num, bb[0][0]]
      newdata7 = pd.concat([newdata7, ii], axis=0)

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", Conve

In [ ]:
# 2020년 7월 예측값
newdata7 = newdata7.iloc[16:,:].reset_index(drop= True)
newdata7

,key,AMT
0,0.0,18.815117
1,0.0,18.804008
2,0.0,18.510878
3,0.0,18.484111
4,0.0,18.813505
...,...,...
13238,1640.0,16.359746
13239,1640.0,16.048576
13240,1640.0,18.504392
13241,1640.0,18.504392


# 4월과 7월 데이터만 뽑기

## 4월 데이터 넣기 (7월 28일 제공)

In [ ]:
# 제공 데이터 인코딩
dtypes = data11_1.dtypes
encoders = {}
for column in data11_1.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(data11_1[column])
        encoders[column] = encoder
        
for column in encoders.keys():
    encoder = encoders[column]
    data11_1[column] = encoder.transform(data11_1[column])

In [ ]:
data11_1

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,0,0,88823988
1,202004,0,1,4708346820
2,202004,0,2,1121028924
3,202004,0,3,14360780
4,202004,0,5,227200
...,...,...,...,...
605,202004,16,36,1373635928
606,202004,16,37,18911036160
607,202004,16,38,14121500
608,202004,16,39,450507431


In [ ]:
# 4월 템플릿 생성
REG_YYMMs     = [202004]
CARD_SIDO_NMs = data2['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = data2['STD_CLSS_NM'].unique()
STD_CLSS_NMs.sort()                                  

temp4 = []
for REG_YYMM in REG_YYMMs:
    for CARD_SIDO_NM in CARD_SIDO_NMs:
        for STD_CLSS_NM in STD_CLSS_NMs:
            temp4.append([REG_YYMM, CARD_SIDO_NM, STD_CLSS_NM])
temp4 = np.array(temp4)
temp4 = pd.DataFrame(data=temp4, columns= ['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])

In [ ]:
temp4

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM
0,202004,0,0
1,202004,0,1
2,202004,0,2
3,202004,0,3
4,202004,0,4
...,...,...,...
692,202004,16,36
693,202004,16,37
694,202004,16,38
695,202004,16,39


In [ ]:
# temp4 생성, 결측치에 0을 넣음
temp4 = pd.merge(temp4, data11_1, how= 'left', left_on = ('REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'), right_on = ('REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'))
temp4 = temp4.fillna(0)

In [ ]:
temp4

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,0,0,8.882399e+07
1,202004,0,1,4.708347e+09
2,202004,0,2,1.121029e+09
3,202004,0,3,1.436078e+07
4,202004,0,4,0.000000e+00
...,...,...,...,...
692,202004,16,36,1.373636e+09
693,202004,16,37,1.891104e+10
694,202004,16,38,1.412150e+07
695,202004,16,39,4.505074e+08


## 7월 데이터 예측

In [ ]:
# 템플릿 만들기
temp7 = daset.loc[daset['key']== 4]
temp7 = temp7.reset_index(drop= True)

In [ ]:
# 7월 예측 코드
for aa in range(17):
  for i in range(41):
    num = num = aa*100 + i
    print(num)
    ii = newdata6.loc[newdata6['key']==num]
    ii = ii.reset_index(drop= True)
    if ii["AMT"][0] == 0:
      print("결측치 있어")
      temp7.loc[num] = [num, 1]
    else:  
      arma = ARIMA(ii["AMT"], order = (0,0,0))
      arima = arma.fit(trend = 'c', full_output = True, disp=1)
      bb = arima.forecast(steps=1)
      print(bb[0][0])
      temp7.loc[num] = [num, bb[0][0]]

0
18.55841055511685
1
21.793198273235067
2
20.814152762692846
3
17.116710802774133
4
결측치 있어
5
15.40016500035211
6
19.952180552092067
7
22.74885596486961
8
14.830350722465155
9
20.79681947506332
10
19.549389905236236
11
22.74315165502464
12
14.023645320195698
13
17.310032316147748
14
4.058395091791016
15
11.999771318497505
16
20.893531767706293
17
20.86981528771855
18
21.5028218564992
19
20.258212788416472
20
23.045994754245356
21
21.74924143757822
22
20.294358843789006
23
18.460079310703946
24
19.12995422166937
25
21.301788951238102
26
20.963144179904127
27
21.32495213904777
28
15.005472422884463
29
19.889139585533297
30
결측치 있어
31
21.064845706527578
32
20.13071180879288
33
23.547327538652176
34
22.974322571578156
35
10.402349084263845
36
21.134585716795
37
23.907418160534505
38
20.426659782462973
39
20.197901010521164
40
21.347232860219066
100
21.053941328608268
101
23.031131343758986
102
22.678177667772097
103
18.655676764042383
104
결측치 있어
105
17.340063826066416
106
22.927273857845
10

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


20.67481296412527
223
13.387978595603663
224
20.803054819419582
225
22.26655174440966
226
22.26213885072432
227
21.59716647206404
228
14.20544650817436
229
18.277315546860226
230
결측치 있어
231
21.66891778679228
232
20.923091117335773
233
24.50166105058308
234
23.70664659389978
235
16.75365246073077
236
22.15142471842106
237
24.768848693766667
238
17.942300534218994
239
21.25953709025779
240
19.713115546502564
300
19.19833680872427
301
22.219084813526234
302
21.375651713574584
303
17.749131589437244
304
3.717723783361495
305
15.356547058555053
306
20.15783223545307
307
23.36469011773231
308
13.894344781420292
309
20.92705878502253
310
19.69625122155077
311
22.836963432900095
312
12.31836722538163
313
16.632673119027448
314
결측치 있어
315
결측치 있어
316
21.39735557754641
317
21.48146605307532
318
21.749803514093166
319
20.91398061842516
320
23.859507911736806
321
22.249745261121117
322
19.968764347591883
323
9.89692469931908
324
20.058929599803147
325
22.313227127176855
326
21.013630250465383
327
2

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


21.262573386259188
332
21.162628225548993
333
24.55791193851882
334
23.28798020131632
335
13.286656439848466
336
21.63281138557962
337
24.380702169258083
338
19.423568060391773
339
20.86503989322041
340
19.10272635816865
400
19.27940496662428
401
19.048537959179594
402
21.314568561158055
403
16.35885267983863
404
결측치 있어
405
15.816111574815077
406
21.14668220679431
407
23.010230471691266
408
11.0847448267134
409
20.61347505963758
410
19.894796198642748
411
20.441082450087414
412
13.6020963763772
413
18.355790022532375
414
결측치 있어
415
16.23243176060732
416
21.515761670689272
417
21.44321129710864
418
21.942341963110252
419
20.659821287185743
420
23.711888686930223
421
21.923450920608868
422
20.043188841631075
423
9.139494541441465
424
19.624533455548683
425
21.80623469555072
426
20.793841682186592
427
20.58626470684149
428
14.996269878663986
429
16.36231627714314
430
결측치 있어
431
21.298987769858847
432
20.574605662301714
433
23.487389068454615
434
22.90418142852778
435
결측치 있어
436
21.4568523

...

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


18.957815441351766
1511
22.659364810577767
1512
12.077787467249241
1513
17.43661486069146
1514
결측치 있어
1515
결측치 있어
1516
21.150805590891
1517
21.422248013650815
1518
21.62841818937942
1519
20.9722792484787
1520
23.359415306991053
1521
21.885496268148803
1522
20.086246957963976
1523
16.085372599850007
1524
19.637647172756687
1525
21.721234828109107
1526
20.709569413887124
1527
21.101986788185005
1528
11.343952974677592
1529
17.586056494866472
1530
11.455555974535859
1531
21.500495238289197
1532
20.578830918979335
1533
24.195003164756972
1534
23.320132884809805
1535
6.529705133883356
1536
21.504061947684825
1537
24.121508818870446
1538
18.277867909596374
1539
20.990264011029627
1540
18.99817495288358
1600
18.929168930026567
1601
21.30610224898512
1602
20.408023400473844
1603
16.850073894311187
1604
결측치 있어
1605
15.291961724445475
1606
21.33024852266093
1607
22.820269251829814
1608
14.534571435212596
1609
20.877259214858242
1610
19.07404087863619
1611
21.882350177904524
1612
결측치 있어
1613
16.7

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


20.968923750559227
1618
21.1384853209088
1619
19.277869946352702
1620
23.188099995218625
1621
21.54609159991129
1622
19.41109250664712
1623
결측치 있어
1624
19.158166634594558
1625
21.5593277603922
1626
20.180522592025014
1627
20.526494678746825
1628
11.752067111080603
1629
16.950967849837262
1630
결측치 있어
1631
21.06314038663739
1632
20.310669851203503
1633
23.83964636392522
1634
22.839363698789313
1635
12.263323236174713
1636
21.068642439510583
1637
23.790599526096443
1638
17.25463540819719
1639
20.20364719001603
1640
18.50439248378753


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [ ]:
# 7월 예측
temp7

,key,AMT
0,0.0,18.558411
1,1.0,21.793198
2,2.0,20.814153
3,3.0,17.116711
4,4.0,1.000000
...,...,...
1636,1636.0,21.068642
1637,1637.0,23.790600
1638,1638.0,17.254635
1639,1639.0,20.203647


In [ ]:
# 로그값을 지수값으로 변환
temp7["AMT"] = np.expm1(temp7["AMT"])

In [ ]:
temp7

,key,AMT
0,0.0,1.147665e+08
1,1.0,2.915183e+09
2,2.0,1.095145e+09
3,3.0,2.714520e+07
4,4.0,1.718282e+00
...,...,...
1636,1636.0,1.412522e+09
1637,1637.0,2.148454e+10
1638,1638.0,3.115968e+07
1639,1639.0,5.947473e+08


# 데이콘 제출

In [ ]:
# 4월과 7월 합치기
dacon =  pd.concat([temp4, temp7], axis=0)
dacon = dacon.reset_index(drop= True)

In [ ]:
dacon

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,key
0,202004.0,0.0,0.0,8.882399e+07,NaN
1,202004.0,0.0,1.0,4.708347e+09,NaN
2,202004.0,0.0,2.0,1.121029e+09,NaN
3,202004.0,0.0,3.0,1.436078e+07,NaN
4,202004.0,0.0,4.0,0.000000e+00,NaN
...,...,...,...,...,...
1389,NaN,NaN,NaN,1.412522e+09,1636.0
1390,NaN,NaN,NaN,2.148454e+10,1637.0
1391,NaN,NaN,NaN,3.115968e+07,1638.0
1392,NaN,NaN,NaN,5.947473e+08,1639.0


In [ ]:
# 반올림
dacon['AMT'] = np.round(dacon["AMT"], 0)

In [ ]:
# 제출 데이터 불러오기
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/여름방학 세미나/input/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)

In [ ]:
# 제출 데이터랑 합치기
submission =  pd.concat([submission, dacon["AMT"]], axis=1)
submission.index.name = 'id'

In [ ]:
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,8.882399e+07
1,202004,강원,골프장 운영업,4.708347e+09
2,202004,강원,과실 및 채소 소매업,1.121029e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.436078e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.412522e+09
1390,202007,충북,한식 음식점업,2.148454e+10
1391,202007,충북,호텔업,3.115968e+07


In [ ]:
# 파일 내보내기
submission.to_csv( '/content/drive/My Drive/Colab Notebooks/여름방학 세미나/input/ARIMA_submission_7.csv', encoding='utf-8-sig')
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,8.882399e+07
1,202004,강원,골프장 운영업,4.708347e+09
2,202004,강원,과실 및 채소 소매업,1.121029e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.436078e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.412522e+09
1390,202007,충북,한식 음식점업,2.148454e+10
1391,202007,충북,호텔업,3.115968e+07


# pyramid Auto Arima의 오류

In [ ]:
a = newdata6.loc[newdata6['key']==0]
a = a["AMT"]
a

0     18.815117
1     18.804008
2     18.510878
3     18.484111
4     18.813505
5     18.400071
6     18.342781
7     18.410864
8     19.140244
9     18.568391
10    18.401341
11    18.371830
12    19.035914
13    18.152874
14    18.380473
15    18.575493
16    18.575493
17    18.575493
Name: AMT, dtype: float64

In [ ]:
from pyramid.arima import auto_arima
auto_arima(a)

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


AttributeError: ignored

In [ ]:
pip install pyramid

# ARIMA X

In [ ]:
data2

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,0,0,148354746
1,201901,0,1,1229200312
2,201901,0,2,1286979106
3,201901,0,3,30656770
4,201901,0,5,13101500
...,...,...,...,...
10043,202004,16,36,1373635928
10044,202004,16,37,18911036160
10045,202004,16,38,14121500
10046,202004,16,39,450507431
